In [50]:
import librosa
import numpy as np
import pandas as pd
from librosa import display
from scipy.io import wavfile
from sklearn.svm import SVC
from scipy.signal import spectrogram
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, GlobalAvgPool1D, Dropout, BatchNormalization, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.utils import np_utils
from keras.regularizers import l2
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from scipy.signal import decimate
from sklearn.model_selection import train_test_split
import glob
import itertools
import numpy as np
import pandas as pd

In [51]:
#read the data from the csv file
heart_data = pd.read_csv('set_a.csv')
heart_data.head()

,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN


In [52]:
#preprocessing to get the value of the data into the list and make ready for classification
artifact=list()
extrahls=list()
murmurs=list()
normal=list()
unlabelled=list()
beats_val=list()
labels_val=list()
fi_names=list()
count=0
for i in range(len(heart_data)):
    if(heart_data['label'][i]=='extrastole'):
        audio, sample_rate = librosa.load(heart_data['fname'][i], res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('extrastole')
        artifact.append(mfccsscaled)
    elif(heart_data['label'][i]=='extrahls'):
        audio, sample_rate = librosa.load(heart_data['fname'][i], res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('extrahls')
        extrahls.append(mfccsscaled)
    elif(heart_data['label'][i]=='murmur'):
        audio, sample_rate = librosa.load(heart_data['fname'][i], res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('murmur')
        murmurs.append(mfccsscaled)
    elif(heart_data['label'][i]=='normal'):
        audio, sample_rate = librosa.load(heart_data['fname'][i], res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        beats_val.append(mfccsscaled)
        labels_val.append('normal')
        normal.append(mfccsscaled)
    else:
        count+=1
        name='Aunlabelledtest'
        tmp=heart_data['fname'][i]
        val=tmp[:6]+name+tmp[6:]
        fi_names.append(val)
        #audio, sample_rate = librosa.load(val, res_type='kaiser_fast') 
        #mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        #mfccsscaled = np.mean(mfccs.T,axis=0)
        #beats_val.append(mfccsscaled)
        #labels_val.append('Unassigned')
        #beats_val.append(data)
        #labels_val.append('NaN')
        


In [41]:
print("Files",end=" ")
print(len(beats_val))

Files 84


In [42]:
# Convert features and corresponding classification labels into numpy arrays
X=np.array(beats_val)
y=np.array(labels_val)

In [43]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [44]:
from sklearn.model_selection import train_test_split 
"""
for SVM
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
clf = SVC()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
print("Accuracy %.3f" % accuracy_score(y_test, predictions))
"""
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [45]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))


In [46]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [47]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 771       
__________

In [48]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 300
num_batch_size = 32

start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 67 samples, validate on 17 samples
Epoch 1/300
67/67 [==============================] - 1s 12ms/step - loss: 10.7075 - acc: 0.2985 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 2/300
67/67 [==============================] - 0s 442us/step - loss: 10.0480 - acc: 0.3582 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 3/300
67/67 [==============================] - 0s 403us/step - loss: 9.9789 - acc: 0.3582 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 4/300
67/67 [==============================] - 0s 378us/step - loss: 10.5850 - acc: 0.3433 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 5/300
67/67 [==============================] - 0s 380us/step - loss: 10.2704 - acc: 0.3582 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 6/300
67/67 [==============================] - 0s 368us/step - loss: 10.3444 - acc: 0.3582 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 7/300
67/67 [==============================] - 0s 374us/step - loss: 10.0333 - acc: 0.3731 - val_loss: 9.4812 - val_acc: 0.4118
Epoch 8/300
67/67 [=

Epoch 122/300
67/67 [==============================] - 0s 398us/step - loss: 7.6807 - acc: 0.5075 - val_loss: 4.7603 - val_acc: 0.7059
Epoch 123/300
67/67 [==============================] - 0s 343us/step - loss: 8.2899 - acc: 0.4776 - val_loss: 5.9780 - val_acc: 0.5882
Epoch 124/300
67/67 [==============================] - 0s 391us/step - loss: 8.5189 - acc: 0.4627 - val_loss: 5.9794 - val_acc: 0.5882
Epoch 125/300
67/67 [==============================] - 0s 378us/step - loss: 6.9555 - acc: 0.5373 - val_loss: 6.5033 - val_acc: 0.5882
Epoch 126/300
67/67 [==============================] - 0s 358us/step - loss: 7.1739 - acc: 0.5224 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 127/300
67/67 [==============================] - 0s 413us/step - loss: 6.6899 - acc: 0.5672 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 128/300
67/67 [==============================] - 0s 350us/step - loss: 7.8854 - acc: 0.5075 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 129/300
67/67 [==============================] - 

Epoch 243/300
67/67 [==============================] - 0s 367us/step - loss: 7.3914 - acc: 0.5224 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 244/300
67/67 [==============================] - 0s 414us/step - loss: 7.2616 - acc: 0.5373 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 245/300
67/67 [==============================] - 0s 405us/step - loss: 7.2036 - acc: 0.5522 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 246/300
67/67 [==============================] - 0s 364us/step - loss: 7.2171 - acc: 0.5522 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 247/300
67/67 [==============================] - 0s 380us/step - loss: 6.8185 - acc: 0.5672 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 248/300
67/67 [==============================] - 0s 351us/step - loss: 6.9782 - acc: 0.5672 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 249/300
67/67 [==============================] - 0s 437us/step - loss: 7.5548 - acc: 0.5224 - val_loss: 6.6369 - val_acc: 0.5882
Epoch 250/300
67/67 [==============================] - 

In [49]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.5970149262627559
Testing Accuracy:  0.5882353186607361


In [56]:
filename = 'set_a/Aunlabelledtest__201108011116.wav' 
audio_data, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
mfccsscaled = np.mean(mfccs.T,axis=0)
predicted_vector = model.predict_classes(np.array([mfccsscaled]))
predicted_class = le.inverse_transform(predicted_vector) 
print("The predicted class is:", predicted_class[0], '\n') 
#use the filenames in fi_values to predict the class label for the unlabelled audio

The predicted class is: murmur 



/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
